In [1]:
import scipy.stats
import matplotlib.pyplot as plt
import copy
from sklearn import linear_model

import matplotlib
import matplotlib.patches as mpatches
from scipy import stats
from scipy.optimize import curve_fit
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import seaborn as sns
import json
import statsmodels.api as sm

import pandas as pd
import numpy as np
import folium
from folium.features import DivIcon
import requests
from bs4 import BeautifulSoup
import re
import squarify
import os

/Users/manavdutta1/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
os.chdir("congressional-districts2018")

FileNotFoundError: [Errno 2] No such file or directory: 'congressional-districts2018'

In [ ]:
#throws exception
congressionalList = []
for dirname, dirnames, filenames in os.walk('.'):
      for subdirname in dirnames:
        if (subdirname != ".git" and subdirname != ".ipynb_checkpoints"):
           with open(os.getcwd() + "/" + subdirname + "/" + "shape.geojson", "r") as f:
                jsonFile = json.load(f)
                district = jsonFile["properties"]["District"]
                if (district.split(" ")[1] != "At-Large"):
                    district = district[0:len(district)-2]
                    jsonFile["properties"]["District"] = district
                print(district)
                congressionalList.append(jsonFile)

In [ ]:
jsonDict = {"type": "FeatureCollection", "features": congressionalList}
with open('geocounties.json', 'w') as outfile:  
    json.dump(jsonDict, outfile)

In [3]:
cookData = pd.read_csv('cookPVI.csv')
for i in range(len(cookData)):
    pvi = cookData.iloc[i]["PVI"]
    if (pvi[0:2] == "D+"):
       value = int(pvi.split("+")[1])     
       cookData.at[i, "PVI"] = value
    elif (pvi == "EVEN"):
       cookData.at[i, "PVI"] = 0
    else:
        value = -1 * int(pvi.split("+")[1])
        cookData.at[i, "PVI"] = value

In [8]:
for i in range(len(cookData)):
    district = cookData.iloc[i]["District"]
    stateArr = district.split(" ")
    if (stateArr[1] == "AL"):
        stateArr[1] = "At-Large"
    cookData.at[i, "District"] = " ".join(stateArr)

In [9]:
print(cookData)

             District  PVI
0           Alabama 1  -15
1           Alabama 2  -16
2           Alabama 3  -16
3           Alabama 4  -30
4           Alabama 5  -18
5           Alabama 6  -26
6           Alabama 7   20
7     Alaska At-Large   -9
8           Arizona 1   -2
9           Arizona 2   -1
10          Arizona 3   13
11          Arizona 4  -21
12          Arizona 5  -15
13          Arizona 6   -9
14          Arizona 7   23
15          Arizona 8  -13
16          Arizona 9    4
17         Arkansas 1  -17
18         Arkansas 2   -7
19         Arkansas 3  -19
20         Arkansas 4  -17
21       California 1  -11
22       California 2   22
23       California 3    5
24       California 4  -10
25       California 5   21
26       California 6   21
27       California 7    3
28       California 8   -9
29       California 9    8
..                ...  ...
405        Virginia 4   10
406        Virginia 5   -6
407        Virginia 6  -13
408        Virginia 7   -6
409        Virginia 8   21
4

In [7]:
retiringDistricts = pd.read_csv("retiring.csv")
retiringDistricts.head()

,District
0,Arizona 2
1,California 39
2,California 49
3,Florida 6
4,Florida 15


In [16]:
geo = r'geocounties.json'
def constructMap(partisan_lean, incumbency_value=3):
    disposableCookData = cookData.copy(deep=True)
    demCount = 0
    for i in range(len(disposableCookData)):
        pvi = disposableCookData.iloc[i]["PVI"]
        pvi = pvi + partisan_lean
        
        if (disposableCookData.iloc[i]["District"] not in retiringDistricts["District"]):
            pvi = pvi + incumbency_value
            
        disposableCookData.at[i, "PVI"] = pvi
        if (disposableCookData.iloc[i]["PVI"] > 0):
            demCount += 1
            
    print("Democratic Count: " + str(demCount))
    
    foliumMap = folium.Map(location=[48, -102], zoom_start=3) 
    foliumMap.choropleth(geo_data=geo, data=disposableCookData,
             columns=["District",'PVI'],
             key_on='feature.properties.District', threshold_scale=[-10, -5, 0, 5, 10],
             fill_color='RdBu', fill_opacity=0.7, line_opacity=0.2,
             legend_name="PVI")
    theMap.save("map.html")
    return foliumMap

In [14]:
#theMap = constructMap(7.5, 3)
#theMap.save("map.html")

In [17]:
slider = widgets.IntSlider(value=0, min=-20, max=20, step=1)
incumbencySlider = widgets.IntSlider(value=0, min=-10, max=10, step=1)
interact(constructMap, partisan_lean=slider)

interactive(children=(IntSlider(value=0, description='partisan_lean', max=20, min=-20), IntSlider(value=3, des…

<function __main__.constructMap>